In [1]:
import tensorflow as tf
import os
import io
import sys
import time
import math
import zipfile
import cv2
import numpy as np
import tensorflow as tf
import six.moves.urllib as urllib
from IPython.display import clear_output, Image, display, HTML
from object_detection.utils import label_map_util
from object_detection.utils.visualization_utils import visualize_boxes_and_labels_on_image_array


In [3]:

MODEL_NAME = 'faster_rcnn_inception_v2_coco_2017_11_08'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'data/mscoco_label_map.pbtxt'
NUM_CLASSES = 90

In [29]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [4]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [5]:
def img2arr(path):
    im = cv2.imread(path)
    img = cv2.cvtColor(cv2.resize(im, (600, 600)), cv2.COLOR_BGR2RGB)
    return img

def arr2img(a, fmt='png'):
    cv2.imshow("Color Image",a[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [6]:
psize = 600
inference_graph = tf.Graph()
with inference_graph.as_default():
    image_tensor = tf.placeholder(tf.float32, shape=(None, psize, psize, 3), name='image_tensor')
    inference_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        inference_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(inference_graph_def, name='',
                            input_map={'Preprocessor/map/TensorArrayStack/TensorArrayGatherV3:0':image_tensor})

In [14]:
def plot_detections(img, scores=None, bboxes=None, min_threshold=0):
    if scores is None or bboxes is None:
        
        inference_sess = tf.Session(graph=inference_graph)

        tensors = [ 
                    inference_graph.get_tensor_by_name('num_detections:0'),
                    inference_graph.get_tensor_by_name('SecondStagePostprocessor/Reshape_4:0'),
                    inference_graph.get_tensor_by_name('SecondStagePostprocessor/convert_scores:0') ]

        feed_dict = { inference_graph.get_tensor_by_name('image_tensor:0'): np.expand_dims(img, axis=0) }

        num_detections, bboxes, scores = inference_sess.run(tensors,
                                                                                                 feed_dict)
        
        bboxes = bboxes[0]
        scores = scores[0]
        
    sorted_classes = np.argsort(scores[:, 1:], axis=1)
    sorted_scores = scores[:, 1:].copy()
    sorted_bboxes = bboxes.copy()

    for i, ordering in enumerate(sorted_classes):
        sorted_scores[i, :] = scores[i, ordering+1]
        sorted_bboxes[i, :] = bboxes[i, ordering, :]

    sorted_classes += 1

    img_viz = visualize_boxes_and_labels_on_image_array(img.copy(), 
                                                        sorted_bboxes[:, -1, :],
                                                        sorted_classes[:, -1].astype(np.int32),
                                                        sorted_scores[:, -1],
                                                        category_index,
                                                        use_normalized_coordinates=False,
                                                        max_boxes_to_draw=sorted_scores.shape[1],
                                                        min_score_thresh=min_threshold,
                                                        line_thickness=1)
    arr2img(img_viz)

img = img2arr('data/stop-sign-2.png')
plot_detections(img, min_threshold=0.9)